# Anime_Rec Live Demo
Here is my working model, suitable for a live demo. 

In [1]:
!nvidia-smi


Fri Sep  8 16:40:53 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.69                 Driver Version: 384.69                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 20%   37C    P8    18W / 250W |     10MiB / 11170MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import mal_scraper
import numpy as np

In [3]:

from keras import backend as K
from keras.layers import Input, Embedding, merge
import keras.layers
from keras.regularizers import l2, l1
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.models import Sequential, Model
from keras.optimizers import SGD, RMSprop, Adam
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
#import pydotplus as pydot 
#import graphviz
from keras.utils import plot_model
from sklearn.metrics import mean_absolute_error

Using TensorFlow backend.


In [4]:
import tensorflow as tf #this code snippit restricts the model from using too much gpu so the recommender_status variation can also be in memory)
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

In [5]:
knownscores  = pd.read_csv('mal_scores_train_nonzero_v2.csv')
known_no_scores= pd.read_csv('mal_scores_train_zero.csv')
anime_avgs = np.load('anime_avgs.npy').item()
anime_names = pd.read_csv('Data/animeinfo3.csv').set_index('animeid')['name'].to_dict()
review_lf_numbers = pd.read_csv('lf_sim_animeid.csv',index_col='Unnamed: 0')
user_review_numbers = pd.read_csv('user_review_scaled_prod.csv',index_col='Unnamed: 0')
review_nlp_numbers = pd.read_csv('reviewnlpsim_animeid.csv',index_col='Unnamed: 0')

In [6]:
userid2idx=np.load("user.npy").item()
animeid2idx=np.load("anime.npy").item()
anime_rev_dict=np.load("anime_rev_number.npy").item()
n_animes = len(animeid2idx)
n_users= len(userid2idx)


In [7]:
n_highest_rated_shows= 5
k_nearest_shows = 7
t_highest_predicted_score=10
l_max_reviews = 50000

In [8]:
#user_to_rec=input("Mal Username to check") # we ask the which user we are looking at

In [9]:
user_to_rec = 'TheBaronVK'

In [10]:
user_scores = knownscores[knownscores['userid']==user_to_rec] # This is the production code

In [11]:
userratinglist2 = []
if user_scores.shape[0]==0: # This means the user doesn't exist in our dataset and we need to scrape him. 
    reviewdict = {'userid':user_to_rec,'name':user_to_rec,'index':user_to_rec}
    useranime = mal_scraper.get_user_anime_list(user_to_rec)
    if useranime is None:
        print('User Has Not reviewed enough anime') # TODO Add seed shows
        pass
    else:
         for animeiter in range(len(useranime)):
            reviewdict = {'name':user_to_rec,'index':user_to_rec}
            reviewdict['unkey']= str(useranime[animeiter]['id_ref'])+str(user_to_rec)
            statusval = useranime[animeiter]['consumption_status'].value
            reviewdict['animescore'] =useranime[animeiter]['score']
            reviewdict['animeid'] =  str(useranime[animeiter]['id_ref'])
            reviewdict['status']= statusval
            userratinglist2.append(reviewdict)
    user_data= pd.DataFrame(userratinglist2) # this is the user's 0 and nonzero scores
    user_data['score']=user_data['animescore'] #standardize the column names
    user_scores=user_data[user_data['score']!=0] # scores
    user_zeroes =user_data[user_data['score']==0] #zero scores
    user_zeroes['animeid']=user_zeroes['animeid'].astype(int)
    useravg= user_scores['score'].mean()
    #print(user_scores.head())
    user_scores['score_usr_scaled']=user_scores['score']-useravg
    #print(user_scores.head())
    user_scores['animeid']=user_scores['animeid'].astype(int)
    user_scores['animeavg']= user_scores.animeid.map(anime_avgs)
    #print(user_scores.head())
    user_scores['score_anime_scaled']=user_scores['score']-user_scores['animeavg']
    #print(user_scores.head())
else:
    user_zeroes=known_no_scores[known_no_scores['userid']==user_to_rec]


ERROR:root:Unable to parse the date text "06-28-15" from an anime list
ERROR:root:Unable to parse the date text "01-13-15" from an anime list
ERROR:root:Unable to parse the date text "06-14-15" from an anime list
ERROR:root:Unable to parse the date text "06-14-15" from an anime list
ERROR:root:Unable to parse the date text "12-21-14" from an anime list
ERROR:root:Unable to parse the date text "07-22-14" from an anime list
ERROR:root:Unable to parse the date text "09-20-14" from an anime list
ERROR:root:Unable to parse the date text "06-14-15" from an anime list
ERROR:root:Unable to parse the date text "06-14-15" from an anime list
ERROR:root:Unable to parse the date text "07-22-14" from an anime list
ERROR:root:Unable to parse the date text "07-22-14" from an anime list
ERROR:root:Unable to parse the date text "06-26-15" from an anime list
ERROR:root:Unable to parse the date text "12-28-14" from an anime list
ERROR:root:Unable to parse the date text "12-15-14" from an anime list
ERROR:

In [12]:
possiblerecs=[]
#user_review_numbers = pd.read_csv('user_review_scaled_prod.csv',index_col='Unnamed: 0')
seedshows = set(user_scores.sort_values('score_usr_scaled',ascending=False)[:n_highest_rated_shows].animeid.values).union(set(user_scores.sort_values('score_anime_scaled',ascending=False)[:n_highest_rated_shows].animeid.values))

In [13]:
#This is the code block to examine seed shows. 
#print("seed shows are:")
#for rec in seedshows:
#    print(anime_names[rec])

In [14]:
possiblerecs1=[]
for show in seedshows:
    topn =list(user_review_numbers.loc[:,str(show)].sort_values(ascending=False)[1:k_nearest_shows+1].index)
    #print(show)
    possiblerecs1 = possiblerecs1+topn
    #print(type(show))

In [15]:
#len(possiblerecs1)#so now we have one third of the target shows to check. 

In [16]:
#review_nlp_numbers = pd.read_csv('reviewnlpsim_animeid.csv',index_col='Unnamed: 0')

In [17]:
#review_nlp_numbers.head()

In [18]:
possiblerecs2 = []
for show in seedshows:
    try:
        topk =list(review_nlp_numbers.loc[:,str(show)].sort_values(ascending=False)[1:k_nearest_shows+1].index)
        possiblerecs2 = possiblerecs2+topk
    except:
        continue
    #print(type(show))

In [19]:
#review_lf_numbers = pd.read_csv('lf_sim_animeid.csv',index_col='Unnamed: 0')

In [20]:
possiblerecs3 = []
for show in seedshows:
    try:
        topk =list(review_lf_numbers.loc[:,str(show)].sort_values(ascending=False)[1:k_nearest_shows+1].index)
        possiblerecs3 = possiblerecs3+topk
    except:
        continue
    #print(type(show))

In [21]:

user_known_anime = list(user_scores.animeid.values)+ list(user_zeroes.animeid.values)

In [22]:
possiblerecs1=[anime for anime in possiblerecs1 if anime not in user_known_anime ]
possiblerecs2=[anime for anime in possiblerecs2 if anime not in user_known_anime ]
possiblerecs3=[anime for anime in possiblerecs3 if anime not in user_known_anime ]
possiblerecs = possiblerecs1+possiblerecs2+possiblerecs3

In [23]:
possiblerecs= list(set(possiblerecs))

In [24]:
possiblerecs= list(set(possiblerecs))

In [25]:
#print("Possible recs are:")
#for rec in possiblerecs:
#    print(anime_names[rec])
    

In [26]:
print("Possible recs1(User Scores) are:")
for rec in set(possiblerecs1):
    print(anime_names[rec])

Possible recs1(User Scores) are:
Zan Sayonara Zetsubou Sensei Bangaichi
Goku Sayonara Zetsubou Sensei
Angel Beats!
Texhnolyze
Zoku Sayonara Zetsubou Sensei
Kizumonogatari I: Tekketsu-hen
Another: The Other - Inga
Sakurasou no Pet na Kanojo
Mousou Dairinin
Baka to Test to Shoukanjuu Ni!
Sora no Otoshimono
Mobile Police Patlabor 2: The Movie
Nekomonogatari: Kuro
Kore wa Zombie Desu ka? of the Dead OVA
Clannad: After Story
Sora no Otoshimono: Forte
Kore wa Zombie Desu ka? OVA
Elfen Lied
Sayonara Zetsubou Sensei Special
Zan Sayonara Zetsubou Sensei
Yojouhan Shinwa Taikei
Yonimo Osoroshii Grimm Douwa
Jiok
Mirai Nikki (TV)
Neko Machi


In [27]:
print("Possible recs2(Tf-IDF) are:")
for rec in set(possiblerecs2):
    print(anime_names[rec])

Possible recs2(Tf-IDF) are:
K
Kyoukai no Kanata Movie: I'll Be Here - Mirai-hen
Haiyore! Nyaruko-san
Tonari no Kaibutsu-kun
Angel Beats!
Ore no Nounai Sentakushi ga, Gakuen Love Comedy wo Zenryoku de Jama Shiteiru
Ookami to Koushinryou
Clannad
Kimi ni Todoke
Ghost Hunt
Zero no Tsukaima
Yuuki Yuuna wa Yuusha de Aru
Hidan no Aria
Gokukoku no Brynhildr
Steins;Gate Movie: Fuka Ryouiki no Déjà vu
Hentai Ouji to Warawanai Neko.
Sakurasou no Pet na Kanojo
Mousou Dairinin
Baka to Test to Shoukanjuu
Baccano!
Nisekoi
Princess Tutu
Flip Flappers
Date A Live II
Panty & Stocking with Garterbelt
Date A Live
Mawaru Penguindrum
Elfen Lied
FLCL
Golden Time
Cardcaptor Sakura
Ao Haru Ride
Fate/kaleid liner Prisma☆Illya
Genei wo Kakeru Taiyou
Kaiba
Kekkai Sensen
Fruits Basket
Mahou Shoujo Lyrical Nanoha: The Movie 1st
Kanon (2006)


In [28]:
print("Possible recs3(Hidden Factors) are:")
for rec in set(possiblerecs3):
    print(anime_names[rec])

Possible recs3(Hidden Factors) are:
Busou Renkin
Dakara Boku wa, H ga Dekinai.
Kantai Collection: KanColle
Shinryaku!? Ika Musume
Bleach
Tonari no Kaibutsu-kun
Kimi ga Nozomu Eien
Working'!!
Ore no Kanojo to Osananajimi ga Shuraba Sugiru
Good Morning Call
Gabriel DropOut Specials
Zero no Tsukaima
Kennosuke-sama
Fairy Tail
Escha & Logy no Atelier: Tasogare no Sora no Renkinjutsushi
Zero no Tsukaima: Futatsuki no Kishi
Owarimonogatari 2nd Season
ef: A Tale of Melodies.
Kimi ni Todoke 2nd Season
D-Frag!
Ao no Exorcist
Shingeki no Kyojin Picture Drama
Kami nomi zo Shiru Sekai: Megami-hen
Ore no Imouto ga Konnani Kawaii Wake ga Nai
Nikutai Ten'i
Danganronpa: Kibou no Gakuen to Zetsubou no Koukousei The Animation
Nisekoi
Tengen Toppa Gurren Lagann
Sukitte Ii na yo.
Shimoneta to Iu Gainen ga Sonzai Shinai Taikutsu na Sekai
Ansatsu Kyoushitsu (TV) 2nd Season: Kagaijugyou-hen
Joshikousei: Girl's High
Elfen Lied
Fate/stay night
Fate/kaleid liner Prisma☆Illya
Million Doll
Rosario to Vampire Capu2

In [29]:
#ratings_score_test = pd.read_csv('mal_scores_test_nonzero_v2.csv')

In [30]:
#ratings_score_test.head()

In [31]:
recs=pd.DataFrame(possiblerecs, columns=['animeid'])

In [32]:
recs['userid']=user_to_rec

In [33]:
recs['animescore']=0
recs['score_anime_scaled']=0
recs['score_usr_scaled']=0


In [34]:
#recs.head()

In [35]:
#type(userid2idx)

In [36]:
#userid2idx

In [37]:
recs['user_id_emb'] = len(userid2idx)+2


In [38]:
user_scores['anime_id_emb']= user_scores.animeid.apply(lambda x: animeid2idx[x])
user_scores['user_id_emb']= len(userid2idx)+1

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [39]:
user_scores['anime_id_emb']= user_scores.animeid.apply(lambda x: animeid2idx[x])
recs['anime_id_emb'] = recs.animeid.apply(lambda x: animeid2idx[x])



/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
n_factors =36

In [41]:
def embedding_input_anime1(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_Anime_Hidden_Factors')(inp)
def embedding_input_user1(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_User_Hidden_Factors')(inp)

user_in1, u1 = embedding_input_user1('user_id_in', n_users+15, n_factors, 0)
u1.trainable=True
anime_in1, a1 = embedding_input_anime1('anime_id_in', n_animes, n_factors, 0)
a1.trainable=False

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(230977, 36, embeddings_regularizer=<keras.reg..., input_length=1, name="Embed_User_Hidden_Factors")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(12873, 36, embeddings_regularizer=<keras.reg..., input_length=1, name="Embed_Anime_Hidden_Factors")`
  This is separate from the ipykernel package so we can avoid doing imports until


In [42]:
# nn1, It is given no modifications for status. It is predicting score
x = merge([u1, a1], mode='concat', name='All_Factors_on_one_layer')
x.trainable=False
x = Flatten()(x)
x.trainable=False
x = Dense(70, activation='relu',name='Random_HF_Interactions')(x)
x.trainable=False
x = Dropout(0.55,name='Prevent_overfit2')(x)
x.trainable=False
x = Dense(16, activation='relu',name='Random_HF_Interactions2')(x)
x.trainable=False
x = Dropout(0.1, name='Prevent_overfit')(x)
x.trainable=False
x = Dense(1,name='Final_Interactions')(x)
x.trainable=False
nn1 = Model([user_in1, anime_in1], x)
nn1.compile(Adam(0.001), loss='mse')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [43]:
nn1.load_weights('nn_score_weights.h5',by_name=True)

In [44]:
#!nvidia-smi

In [45]:
nn1.fit([user_scores.user_id_emb, user_scores.anime_id_emb], user_scores.score, epochs=18)

Epoch 1/18
112/112 [==============================] - 0s - loss: 1.5019     
Epoch 2/18
112/112 [==============================] - 0s - loss: 1.2992     
Epoch 3/18
112/112 [==============================] - 0s - loss: 1.1959     
Epoch 4/18
112/112 [==============================] - 0s - loss: 1.2066     
Epoch 5/18
112/112 [==============================] - 0s - loss: 1.1782     
Epoch 6/18
112/112 [==============================] - 0s - loss: 1.0360     
Epoch 7/18
112/112 [==============================] - 0s - loss: 0.9615     
Epoch 8/18
112/112 [==============================] - 0s - loss: 1.0511     
Epoch 9/18
112/112 [==============================] - 0s - loss: 1.0006     
Epoch 10/18
112/112 [==============================] - 0s - loss: 1.0286     
Epoch 11/18
112/112 [==============================] - 0s - loss: 0.9525     
Epoch 12/18
112/112 [==============================] - 0s - loss: 0.9746     
Epoch 13/18
112/112 [==============================] - 0s - loss: 0.8712 

In [46]:
recs['score']=nn1.predict([recs.user_id_emb, recs.anime_id_emb])

In [47]:
#recs.head()

In [48]:
#recs.sort_values('score',ascending=False)

In [49]:
def embedding_input_anime22(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_Anime_Hidden_Factors')(inp)
def embedding_input_user22(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_User_Hidden_Factors')(inp)

user_in22, u22 = embedding_input_user22('user_id_in', n_users+15, n_factors, 0)
u22.trainable=True
anime_in22, a22 = embedding_input_anime22('anime_id_in', n_animes, n_factors, 0)
a22.trainable=False
# nn22, It is only given complete. Trying to predict usr scaled score. 
x = merge([u22, a22], mode='concat', name='All_Factors_on_one_layer')
x.trainable=False
x = Flatten()(x)
x.trainable=False
x = Dense(70, activation='relu',name='Random_HF_Interactions')(x)
x.trainable=False
x = Dropout(0.55,name='Prevent_overfit2')(x)
x.trainable=False
x = Dense(16, activation='relu',name='Random_HF_Interactions2')(x)
x.trainable=False
x = Dropout(0.1, name='Prevent_overfit')(x)
x.trainable=False
x = Dense(1,name='Final_Interactions')(x)
x.trainable=False
nn22 = Model([user_in22, anime_in22], x)
nn22.compile(Adam(0.001), loss='mse')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(230977, 36, embeddings_regularizer=<keras.reg..., input_length=1, name="Embed_User_Hidden_Factors")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(12873, 36, embeddings_regularizer=<keras.reg..., input_length=1, name="Embed_Anime_Hidden_Factors")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.

In [50]:
nn22.load_weights('nn_score_usr_weights.h5',by_name=True)

In [51]:
nn22.fit([user_scores.user_id_emb, user_scores.anime_id_emb], user_scores.score_usr_scaled, epochs=24)

Epoch 1/24
112/112 [==============================] - 0s - loss: 1.2206     
Epoch 2/24
112/112 [==============================] - 0s - loss: 1.2260     
Epoch 3/24
112/112 [==============================] - 0s - loss: 1.0933     
Epoch 4/24
112/112 [==============================] - 0s - loss: 1.1269     
Epoch 5/24
112/112 [==============================] - 0s - loss: 1.0649     
Epoch 6/24
112/112 [==============================] - 0s - loss: 1.0320     
Epoch 7/24
112/112 [==============================] - 0s - loss: 0.9955     
Epoch 8/24
112/112 [==============================] - 0s - loss: 0.9723     
Epoch 9/24
112/112 [==============================] - 0s - loss: 0.9540     
Epoch 10/24
112/112 [==============================] - 0s - loss: 0.9062     
Epoch 11/24
112/112 [==============================] - 0s - loss: 0.8887     
Epoch 12/24
112/112 [==============================] - 0s - loss: 0.8372     
Epoch 13/24
112/112 [==============================] - 0s - loss: 0.8333 

In [52]:
recs['score_usr_scaled']=nn22.predict([recs.user_id_emb, recs.anime_id_emb])

In [53]:
def embedding_input_anime23(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_Anime_Hidden_Factors')(inp)
def embedding_input_user23(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_User_Hidden_Factors')(inp)

user_in23, u23 = embedding_input_user23('user_id_in', n_users+15, n_factors, 0)
u23.trainable=True
anime_in23, a23 = embedding_input_anime23('anime_id_in', n_animes, n_factors, 0)
a23.trainable=False
# nn22, It is only given complete. Trying to predict usr scaled score. 
x = merge([u23, a23], mode='concat', name='All_Factors_on_one_layer')
x.trainable=False
x = Flatten()(x)
x.trainable=False
x = Dense(70, activation='relu',name='Random_HF_Interactions')(x)
x.trainable=False
x = Dropout(0.55,name='Prevent_overfit2')(x)
x.trainable=False
x = Dense(16, activation='relu',name='Random_HF_Interactions2')(x)
x.trainable=False
x = Dropout(0.1, name='Prevent_overfit')(x)
x.trainable=False
x = Dense(1,name='Final_Interactions')(x)
x.trainable=False
nn23 = Model([user_in23, anime_in23], x)
nn23.compile(Adam(0.001), loss='mse')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(230977, 36, embeddings_regularizer=<keras.reg..., input_length=1, name="Embed_User_Hidden_Factors")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(12873, 36, embeddings_regularizer=<keras.reg..., input_length=1, name="Embed_Anime_Hidden_Factors")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.

In [54]:
nn23.load_weights('nn_score_anime_weights.h5',by_name=True)

In [55]:
nn23.fit([user_scores.user_id_emb, user_scores.anime_id_emb], user_scores.score_usr_scaled, epochs=18)

Epoch 1/18
112/112 [==============================] - 0s - loss: 1.2817     
Epoch 2/18
112/112 [==============================] - 0s - loss: 1.1828     
Epoch 3/18
112/112 [==============================] - 0s - loss: 1.1886     
Epoch 4/18
112/112 [==============================] - 0s - loss: 1.0840     
Epoch 5/18
112/112 [==============================] - 0s - loss: 1.1026     
Epoch 6/18
112/112 [==============================] - 0s - loss: 1.0804     
Epoch 7/18
112/112 [==============================] - 0s - loss: 1.0829     
Epoch 8/18
112/112 [==============================] - 0s - loss: 0.9946     
Epoch 9/18
112/112 [==============================] - 0s - loss: 1.0457     
Epoch 10/18
112/112 [==============================] - 0s - loss: 1.0046     
Epoch 11/18
112/112 [==============================] - 0s - loss: 0.8968     
Epoch 12/18
112/112 [==============================] - 0s - loss: 0.9979     
Epoch 13/18
112/112 [==============================] - 0s - loss: 0.8580 

In [56]:
recs['score_anime_scaled']=nn23.predict([recs.user_id_emb, recs.anime_id_emb])

In [57]:
recs['anime_rev_count']= recs.animeid.map(anime_rev_dict)

In [58]:
recs=recs[recs['anime_rev_count']<l_max_reviews]

In [59]:
#recs.sort_values('score_usr_scaled',ascending=False).head(t_highest_predicted_score)

In [60]:
#recs.sort_values('score_anime_scaled',ascending=False).head(t_highest_predicted_score)

In [61]:
#recs.sort_values('score',ascending=False).head(t_highest_predicted_score)

In [62]:
score_user_preds =recs.sort_values('score_usr_scaled',ascending=False).head(t_highest_predicted_score)
score_preds =recs.sort_values('score',ascending=False).head(t_highest_predicted_score)
score_anime_preds =recs.sort_values('score_anime_scaled',ascending=False).head(t_highest_predicted_score)

In [63]:
#score_preds.animeid.values

In [64]:
print("seed shows are:")
for rec in seedshows:
    print(anime_names[rec])

seed shows are:
Toradora!
Kore wa Zombie Desu ka? of the Dead
Another
Kore wa Zombie Desu ka?
Mahou Shoujo Madoka★Magica Movie 2: Eien no Monogatari
Sayonara Zetsubou Sensei
Boogiepop wa Warawanai: Boogiepop Phantom
Bakemonogatari
Mahou Shoujo Madoka★Magica


In [65]:
print("Possible recs1(User Scores) are:")
for rec in set(possiblerecs1):
    print(anime_names[rec])

Possible recs1(User Scores) are:
Zan Sayonara Zetsubou Sensei Bangaichi
Goku Sayonara Zetsubou Sensei
Angel Beats!
Texhnolyze
Zoku Sayonara Zetsubou Sensei
Kizumonogatari I: Tekketsu-hen
Another: The Other - Inga
Sakurasou no Pet na Kanojo
Mousou Dairinin
Baka to Test to Shoukanjuu Ni!
Sora no Otoshimono
Mobile Police Patlabor 2: The Movie
Nekomonogatari: Kuro
Kore wa Zombie Desu ka? of the Dead OVA
Clannad: After Story
Sora no Otoshimono: Forte
Kore wa Zombie Desu ka? OVA
Elfen Lied
Sayonara Zetsubou Sensei Special
Zan Sayonara Zetsubou Sensei
Yojouhan Shinwa Taikei
Yonimo Osoroshii Grimm Douwa
Jiok
Mirai Nikki (TV)
Neko Machi


In [66]:
print("Possible recs2(Tf-IDF) are:")
for rec in set(possiblerecs2):
    print(anime_names[rec])

Possible recs2(Tf-IDF) are:
K
Kyoukai no Kanata Movie: I'll Be Here - Mirai-hen
Haiyore! Nyaruko-san
Tonari no Kaibutsu-kun
Angel Beats!
Ore no Nounai Sentakushi ga, Gakuen Love Comedy wo Zenryoku de Jama Shiteiru
Ookami to Koushinryou
Clannad
Kimi ni Todoke
Ghost Hunt
Zero no Tsukaima
Yuuki Yuuna wa Yuusha de Aru
Hidan no Aria
Gokukoku no Brynhildr
Steins;Gate Movie: Fuka Ryouiki no Déjà vu
Hentai Ouji to Warawanai Neko.
Sakurasou no Pet na Kanojo
Mousou Dairinin
Baka to Test to Shoukanjuu
Baccano!
Nisekoi
Princess Tutu
Flip Flappers
Date A Live II
Panty & Stocking with Garterbelt
Date A Live
Mawaru Penguindrum
Elfen Lied
FLCL
Golden Time
Cardcaptor Sakura
Ao Haru Ride
Fate/kaleid liner Prisma☆Illya
Genei wo Kakeru Taiyou
Kaiba
Kekkai Sensen
Fruits Basket
Mahou Shoujo Lyrical Nanoha: The Movie 1st
Kanon (2006)


In [67]:
print("Possible recs3(Hidden Factors) are:")
for rec in set(possiblerecs3):
    print(anime_names[rec])

Possible recs3(Hidden Factors) are:
Busou Renkin
Dakara Boku wa, H ga Dekinai.
Kantai Collection: KanColle
Shinryaku!? Ika Musume
Bleach
Tonari no Kaibutsu-kun
Kimi ga Nozomu Eien
Working'!!
Ore no Kanojo to Osananajimi ga Shuraba Sugiru
Good Morning Call
Gabriel DropOut Specials
Zero no Tsukaima
Kennosuke-sama
Fairy Tail
Escha & Logy no Atelier: Tasogare no Sora no Renkinjutsushi
Zero no Tsukaima: Futatsuki no Kishi
Owarimonogatari 2nd Season
ef: A Tale of Melodies.
Kimi ni Todoke 2nd Season
D-Frag!
Ao no Exorcist
Shingeki no Kyojin Picture Drama
Kami nomi zo Shiru Sekai: Megami-hen
Ore no Imouto ga Konnani Kawaii Wake ga Nai
Nikutai Ten'i
Danganronpa: Kibou no Gakuen to Zetsubou no Koukousei The Animation
Nisekoi
Tengen Toppa Gurren Lagann
Sukitte Ii na yo.
Shimoneta to Iu Gainen ga Sonzai Shinai Taikutsu na Sekai
Ansatsu Kyoushitsu (TV) 2nd Season: Kagaijugyou-hen
Joshikousei: Girl's High
Elfen Lied
Fate/stay night
Fate/kaleid liner Prisma☆Illya
Million Doll
Rosario to Vampire Capu2

In [68]:
print("Recommendations for score are:")
for rec in score_preds.animeid.values:
    print(anime_names[rec])

Recommendations for score are:
Owarimonogatari 2nd Season
Yojouhan Shinwa Taikei
Clannad: After Story
Steins;Gate Movie: Fuka Ryouiki no Déjà vu
Tengen Toppa Gurren Lagann
Kizumonogatari I: Tekketsu-hen
Baccano!
Kyoukai no Kanata Movie: I'll Be Here - Mirai-hen
Sakurasou no Pet na Kanojo
Ookami to Koushinryou


In [69]:
print("Recommendations for  User_Scaled score are:")
for rec in score_user_preds.animeid.values:
    print(anime_names[rec])

Recommendations for  User_Scaled score are:
Owarimonogatari 2nd Season
Clannad: After Story
Tengen Toppa Gurren Lagann
Yojouhan Shinwa Taikei
Steins;Gate Movie: Fuka Ryouiki no Déjà vu
Baccano!
Kizumonogatari I: Tekketsu-hen
Clannad
Kyoukai no Kanata Movie: I'll Be Here - Mirai-hen
Sakurasou no Pet na Kanojo


In [70]:
print("Recommendations for  Anime_Scaled score are:")
for rec in score_anime_preds.animeid.values:
    print(anime_names[rec])

Recommendations for  Anime_Scaled score are:
Sayonara Zetsubou Sensei Special
Aimai Elegy
Good Morning Call
Zan Sayonara Zetsubou Sensei Bangaichi
Flip Flappers
Zan Sayonara Zetsubou Sensei
Owarimonogatari 2nd Season
Tengen Toppa Gurren Lagann
Baccano!
Kizumonogatari I: Tekketsu-hen
